In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import sys
import time
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, random_split, DataLoader
import torchvision.transforms as transforms
loader=transforms.ToTensor()

from PIL import Image

In [2]:
import os
import shutil
data_dir=r'../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'
os.listdir(data_dir)

In [3]:
root=r'../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'

IMG_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', 'webp']
def has_file_allowed_extension(filename, extensions):
    """Check if the file is supported scalable type

    Args:
        filename (string): file path 
                 Extensions: Scalable type list, an acceptable image file type

    Returns:
        bool: True if the filename ends with one of given extensions
    """
    filename_lower = filename.lower()
    return any(filename_lower.endswith(ext) for ext in  extensions) # Return to True or False list


def make_dataset(dir, class_to_idx, extensions):
    """
                Return, such as [(image path, category index value corresponding to), (), ...]
    """
    images = []
    dir = os.path.expanduser(dir)
    for target in class_to_idx.keys():
        d=os.path.join(dir,target,target)
        print(d)
        if not os.path.isdir(d):
            continue
        for root, _, fnames in  sorted(os.walk (d)):
            for fname in fnames:
                if  has_file_allowed_extension(fname, extensions): #Viewing if the file is supported, it is continued
                    path = os.path.join(root, fname)
                    item = (path, class_to_idx[target])
                    images.append(item)
    return images
                
    
class CustomDataFolder(Dataset):
    
    def __init__(self, root, loader, extensions, transform=None, target_transform=None):
        
        def find_classes( dir):
            classes=[]
            for class_ in os.listdir(dir):
                if os.path.isdir(root+'/'+class_):
                    classes.append(class_)
            class_to_idx = {classes[i]: i for i in  range(len(classes))}
            return classes, class_to_idx
        
        classes, class_to_idx = find_classes (root) # get class name and class index, such as ['cat', 'dog']with{'cat': 0, 'dog': 1}
                 #       [(image path, the category value of the image), (), ...], that is, tag each image
        samples = make_dataset(root, class_to_idx, extensions) 
        if len(samples) == 0:
            raise(RuntimeError("Found 0 files in subfolders of: " + root + "\n"
                               "Supported extensions are: " + ",".join(extensions)))

        self.root = root
        self.loader = loader
        self.extensions = extensions

        self.classes = classes
        self.class_to_idx = class_to_idx
        self.samples = samples
        self.targets = [s[1] for s in  samples] # Lists consisting of all images

        self.transform = transform
        self.target_transform = target_transform
    
    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        """
        path, target = self.samples[index]
        image = Image.open(path)
        sample =  self.loader (image) # loading pictures
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target
    

    
    def __len__(self):
        return len(self.samples)

    def __repr__(self):
        fmt_str = 'Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
        fmt_str += '    Root Location: {}\n'.format(self.root)
        tmp = '    Transforms (if any): '
        fmt_str += '{0}{1}\n'.format(tmp, self.transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        tmp = '    Target Transforms (if any): '
        fmt_str += '{0}{1}'.format(tmp, self.target_transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        return fmt_str

In [4]:
class CustomImageFolder(CustomDataFolder):
    """A generic data loader where the images are arranged in this way: ::

        root/dog/xxx.png
        root/dog/xxy.png
        root/dog/xxz.png

        root/cat/123.png
        root/cat/nsdf3.png
        root/cat/asd932_.png"""
    def __init__(self, root, transform=None, target_transform=None,
                 loader=loader):
        super(CustomImageFolder, self).__init__(root, loader, IMG_EXTENSIONS,
                                          transform=transform,
                                          target_transform=target_transform)
        self.imgs = self.samples

In [5]:

import torchvision.transforms as tt
transform = tt.Compose([
                         tt.Scale((400,400)), 
                         # tt.RandomRotate
                         # tt.RandomResizedCrop(256, scale=(0.5,0.9), ratio=(1, 1)), 
                         # tt.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
                         ])

dataset=CustomImageFolder(root,transform=transform)

In [6]:
img,label=dataset[5000]
print(img.shape,label)

In [ ]:
dataset.classes

In [9]:
def show_example(img, label):
    print('Label: ', dataset.classes[label], "("+str(label)+")")
    plt.imshow(img.permute(1, 2, 0))
show_example(*dataset[10])

In [ ]:
show_example(*dataset[5000])

In [ ]:
show_example(*dataset[7000])

## Divide Data into train and test

In [7]:
random_seed = 42
torch.manual_seed(random_seed);


In [8]:
val_size=900
train_size=len(dataset)-val_size

train_ds,valid_ds= random_split(dataset, [train_size, val_size])
print(len(train_ds),len(valid_ds))


`Load the data into batches`

In [9]:
from torch.utils.data.dataloader import DataLoader

batch_size=16

In [10]:
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(valid_ds, batch_size, num_workers=2, pin_memory=True)

In [11]:
for img,label in train_dl:
    print(img.shape)
    print(label)
    break

## Show Multiple Images in Grid

In [15]:
from torchvision.utils import make_grid

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=4).permute(1, 2, 0))
        break
show_batch(train_dl)

## GPU settings

In [13]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [14]:
device = get_default_device()
device

## Load data to GPU

In [16]:
train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(val_dl, device)

## Plotting Functions

In [17]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');
    
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

## Base Class Module

In [18]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

## Resnet Architecture

`Resnet9 Architecture`

![](http://raw.githubusercontent.com/lambdal/cifar10-fast/master/net.svg)

## Single Block
![](http://miro.medium.com/max/1140/1*D0F3UitQ2l5Q0Ak-tjEdJg.png)

In [ ]:
class SimpleResidualBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        return self.relu2(out) + x # ReLU can be applied before or after adding the input

## Test the residual block

In [ ]:
simple_resnet = to_device(SimpleResidualBlock(), device)

for images, labels in train_dl:
    out = simple_resnet(images)
    print(out.shape)
    break
    
del simple_resnet, images, labels
torch.cuda.empty_cache()

## Main Resnet Architecture

In [19]:
def conv_block(in_channels,out_channels,pool=False):
    layers=[nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1),
           nn.BatchNorm2d(out_channels),
           nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(ImageClassificationBase):
    def __init__(self,in_channels,num_classes):
        super().__init__()
        self.conv1=conv_block(in_channels,64)
        self.conv2=conv_block(64,128,pool=True)
        self.res1=nn.Sequential(conv_block(128, 128), conv_block(128, 128))
        self.conv3=conv_block(128, 256, pool=True)
        self.conv4=conv_block(256, 512, pool=True)
        self.res2=nn.Sequential(conv_block(512, 512), conv_block(512, 512))
        self.classifier = nn.Sequential(nn.MaxPool2d(4), 
                                        nn.Flatten(), 
                                        nn.Dropout(0.2),
                                        nn.Linear(73728, num_classes))
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out    
    
model = to_device(ResNet9(3, 9), device)
model
# After res2 torch.Size([64, 512, 50, 50])
# After maxpool torch.Size([64, 512, 12, 12])
# torch.Size([64, 73728])
# torch.Size([64, 73728])

In [22]:
torch.cuda.empty_cache()

## Paramters


In [20]:
batch_size = 32
n_iters = 30
epochs  = 10#int( n_iters / (len(train_dl) / batch_size))
input_dim = 3*400*400
output_dim = 9
lr_rate  = 0.001

## Training the Model

In [20]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
for img,label in valid_dl:
    print(img.shape)
    print(label)
    break

In [21]:
history = [evaluate(model, valid_dl)]
history

In [23]:
epochs = 8
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [24]:
%%time
history += fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

In [30]:
plot_accuracies(history)

In [31]:
plot_losses(history)

## Test the images

In [28]:
from torchvision.datasets import ImageFolder
path='../input/a-large-scale-fish-dataset/NA_Fish_Dataset'
import torchvision.transforms as tt
transform = tt.Compose([
                         tt.Scale((400,400)), 
                         tt.ToTensor()
                         # tt.RandomRotate
                         # tt.RandomResizedCrop(256, scale=(0.5,0.9), ratio=(1, 1)), 
                         # tt.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
                         ])
test_data=ImageFolder(path,transform=transform)
test_data.classes

In [29]:
for img,label in test_data:
    print(img.shape)
    print(label)
    break

In [35]:
def predict_image(img,model):
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    # Retrieve the class label
    return test_data.classes[preds[0].item()]


In [44]:
img,label=test_data[34]
plt.imshow(img.permute(1, 2, 0).clamp(0, 1))
print('Label:', test_data.classes[label], ', Predicted:', predict_image(img, model))

## Save the Parameters

In [45]:
torch.save(model.state_dict(), 'fishes_large_scale-resnet9.pth')